#### 양자화된 모델에 대한 문장 생성

1. casllm-base-7b-hf: Meta Llama2 7B 허깅페이스 모델
2. casllm-base-7b-hf-chat: Meta Llama2 7B-Chat 허깅페이스 모델
2. bllossom-llama2-13bhf-chat-4bit: 허깅페이스에서 bllossom 님의 라마 13b Chat 모델을 4bit 양자화 한 한글학습 모델


In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
#model_id = "/data/bwllm/models/bllossom-llama2-13bhf-chat-4bit"
model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bhf-8bit"

# koalpaca-8bit 양자화 모델에서만 적용가능(아래)
#model = LlamaForCausalLM.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id)
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [ ]:
print(model)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text
        
        

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템에서의 감리절차에 대해 알려줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 토크나이저를 사용해 입력 텍스트 인코딩
inputs = tokenizer.encode(prompt_text, return_tensors='pt')

In [ ]:
# 모델을 GPU로 이동
# T4 16G 에서 메모리에 로딩하는 것을 실패함.
#model.to('cuda')

# 입력 데이터도 GPU로 이동
#inputs = inputs.to('cuda')

print(inputs)

In [ ]:
# 모델을 사용해 문장 생성
# num_return_sequences는 반환할 문장 수, max_length는 최대 길이를 설정
output_sequences = model.generate(
    input_ids=inputs,
    max_length=2000,
    num_return_sequences=1,
    temperature=1.0,  # 생성 다양성 조절
)

In [ ]:
print(output_sequences)

In [ ]:
# 생성된 문장을 디코딩하여 출력
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)